In [ ]:
import findspark
findspark.init('/opt/spark')

from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .getOrCreate()

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [ ]:
crimeFacts = (spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("/tmp/crime.csv")
    .cache())

In [ ]:
from pyspark.sql.functions import col, pandas_udf
import pandas as pd
import numpy as np

In [ ]:
crimeFacts = crimeFacts.withColumn("year", col("OCCURRED_ON_DATE").substr(0, 4)) \
       .withColumn("month", col("OCCURRED_ON_DATE").substr(6, 2))

In [ ]:
@pandas_udf("int")
def cumulative_sum_overall(series: pd.Series) -> pd.Series:
    return series.cumsum()

In [ ]:
@pandas_udf("int")
def cumulative_sum_by_year(pdf: pd.DataFrame) -> pd.Series:
    pdf = pdf.sort_values(["year", "month"])
    return pdf.groupby("year")["crime_count"].cumsum()

In [ ]:
crimeFacts = crimeFacts.groupBy("year", "month").count().withColumnRenamed("count", "crime_count")

In [ ]:
crimeFacts = crimeFacts.withColumn("cumulative_sum_overall", cumulative_sum_overall(col("crime_count")))

In [ ]:
crimeFacts = crimeFacts.withColumn("cumulative_sum_by_year", cumulative_sum_by_year(col("crime_count")))

In [ ]:
crimeFacts.show()